In [0]:
import dataiku
import pandas as pd
import numpy as np
from textblob import TextBlob


In [0]:
filtered_tweets = dataiku.Dataset("filtered_tweets")
df = filtered_tweets.get_dataframe()
enhanced_tweets_informations = dataikuy.Dataset("enhanced_tweets_informations")
whole_df = enhanced_tweets_informations.get_dataframe()


In [0]:

def get_sentiment_label(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity <= -0.6:
        return "very negative"
    elif -0.6 < polarity <= -0.2:
        return "negative"
    elif -0.2 < polarity <= 0.2:
        return "neutral"
    elif 0.2 < polarity <= 0.6:
        return "positive"
    else:
        return "very positive"

df['label'] = df['text'].apply(get_sentiment_label)


In [0]:
df.head(10)